<a href="https://colab.research.google.com/github/Debottam/MachinLearningEx/blob/master/LSHash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
vec1 = np.random.rand(5)
vec2 = np.random.rand(5)
vec3 = np.random.rand(5)

In [3]:
vec1, vec2, vec3

(array([0.5451425 , 0.7868748 , 0.82286995, 0.45989078, 0.12072714]),
 array([0.01548088, 0.06736177, 0.8137304 , 0.47010442, 0.02598073]),
 array([0.68912075, 0.70650266, 0.13144711, 0.41443441, 0.99413623]))

In [0]:
projections = np.random.randn(2,5)

In [0]:
np.dot(vec1, projections.T)

array([-0.48402468, -0.38112615])

In [0]:
class HashTable:
    def __init__(self, hash_size, inp_dimensions):
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_table = dict()
        self.projections = np.random.randn(self.hash_size, inp_dimensions)
        
    def generate_hash(self, inp_vector):
        bools = (np.dot(inp_vector, self.projections.T) > 0).astype('int')
        return ''.join(bools.astype('str'))

    def __setitem__(self, inp_vec, label):
        hash_value = self.generate_hash(inp_vec)
        self.hash_table[hash_value] = self.hash_table.get(hash_value, list()) + [label]
        
    def __getitem__(self, inp_vec):
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])

In [0]:
hash_table = HashTable(hash_size=3, inp_dimensions=5)

In [0]:
 hash_table.__setitem__(vec1, 0)

In [0]:
hash_table.__setitem__(vec2, 1)

In [0]:
hash_table.__setitem__(vec3, 2)

In [43]:
hash_table.__getitem__(vec3)

[0, 2]

In [45]:
hash_table.__getitem__(vec2)

[1]